# BASE DE DATOS III - TAREA N°2
**Entorno:** Anaconda (Python)
_Notebook preparado para ejecutarse en un entorno Anaconda Python local._
---

## Resumen de la tarea
**Integrantes:**
- Rodrigo Guerrero
- Miguel Espinoza
---

# Descripción del dataset
Conjunto de datos oficiales que recopila indicadores hospitalarios de establecimientos públicos y privados de Chile, correspondientes a procesos de hospitalización registrados por el Ministerio de Salud (MINSAL).
- **Tamaño original:** 155,339 filas, 20 columnas
- **Origen:** Evaluación empírica con información de un conjunto de datos hospitalario de Chile
---

# Variables del dataset
**Variable objetivo**
- **EFICIENCIA:** Esta columna indica si un hospital es eficiente o no (1 = Sí, 0 = No).

**Variables independientes**
- **TIPO_PERTENENCIA:** Código numérico que identifica la pertenencia del establecimiento (entero)
- **GLOSA_SSS:** Nombre del Servicio de Salud (texto)
- **PERIODO:** Año del registro (entero)
- **ESTABLECIMIENTO:** Nombre del establecimiento (texto)
- **AREA_FUNCIONAL:** Nombre del área funcional (texto)
- **DIAS_CAMAS_OCUPADAS:** Total de las camas ocupadas durante el periodo (entero)
- **DIAS_CAMAS_DISPONIBLES:** Total de días que las camas estuvieron disponibles durante el periodo (entero)
- **DIAS_ESTADA:** Suma de los días de estadía de todos los pacientes hospitalarios durante el periodo (entero)
- **NUMERO_EGRESOS:** Total de pacientes que egresaron del hospital (entero)
- **MES:** Mes que se realizó el registro (entero)
- **EGRESOS_FALLECIDOS:** Número de pacientes que fallecieron durante hospitalización (entero)
- **TRASLADOS:** Cantidad de egresos que corresponden a pacientes trasladados a otro centro (entero)
- **INDICE_OCUPACIONAL:** Proporción de camas ocupadas respecto a las disponibles (decimal)
- **PROMEDIO_CAMAS_DISPONIBLES:** Promedio de camas disponibles (decimal)
- **PROMEDIO_DIAS_ESTADA:** Días que un paciente permanece hospitalizado (decimal)
- **LETALIDAD:** Porcentaje de fallecidos respecto al total de egresos (decimal)
- **INDICE_ROTACION:** Número promedio de egresos por cama durante el periodo (decimal)
- **COD_SSS:** Código numérico que identifica al Servicio de Salud al que pertenece un establecimiento hospitalario (entero)
- **CODIGO_ESTABLECIMIENTO:** Código único que identifica a cada establecimiento de salud dentro del sistema (entero)
- **COD_AREA_FUNCIONAL:** Código numérico que corresponde al área funcional del hospital o centro de salud (entero)
---

### LIBRERÍAS UTILIZADAS

- **pandas**: Manipulación y análisis de datos estructurados
- **numpy**: Operaciones numéricas y arrays multidimensionales
- **matplotlib.pyplot**: Creación de gráficos y visualizaciones
- **seaborn**: Visualizaciones estadísticas avanzadas


In [ ]:
#librerias
import pandas as pd  # Pandas para manipulación de datos
import matplotlib.pyplot as plt  # Matplotlib para visualización de datos
import seaborn as sns  # Seaborn para gráficos estadísticos
import numpy as np  # NumPy para operaciones numéricas
import re

from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier  # K-Vecinos más cercanos
from sklearn.linear_model import LogisticRegression  # Regresión Logística
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix,precision_score  # Métricas de rendimiento
from sklearn.model_selection import train_test_split  # División de datos
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA  # Reducción de dimensionalidad
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV, train_test_split


# **1. a. Descripción utilizando métodos estadísticos**

## **Descripcion detallada del dataset**

In [ ]:
# Cargar el dataset
csv_path = r"indicadores_rem20_20250925.csv"
df = pd.read_csv(csv_path, na_values=["nan ", ""], thousands=",", quotechar='"', on_bad_lines='skip', delimiter=';')

# Ver las primeras filas
# Identificar los tipos de datos de cada columna
print(df.dtypes)

# Descripción estadística de variables numéricas
print(df.describe())

## **Variable Eficiencia**

**Justificación de uso**

- La variable EFICIENCIA se construye como un indicador compuesto que refleja el desempeño operativo y clínico de los hospitales incluidos en el dataset. Se consideraron tres dimensiones fundamentales:

- Índice Ocupacional (INDICE_OCUPACIONAL):
Representa la proporción de camas efectivamente utilizadas en relación con la capacidad total. Se define un umbral de 0.7, considerando que un hospital que mantiene más del 70% de ocupación logra un uso adecuado de sus recursos, evitando subutilización excesiva de camas y personal.


- Letalidad (LETALIDAD):
Se mide como la proporción de pacientes fallecidos sobre el total de egresos. Para efectos de eficiencia, se considera favorable un nivel inferior al 5% (0.05), asumiendo que hospitales con menor letalidad combinan atención oportuna y calidad clínica.


- Índice de Rotación (INDICE_ROTACION):
Este índice refleja la velocidad con que las camas se desocupan y se ocupan nuevamente, indicando la capacidad de gestión de flujos de pacientes. Se establece que una eficiencia mayor se asocia a valores superiores a la mediana del dataset, lo que implica un manejo más dinámico y eficiente de los recursos hospitalarios.

In [ ]:
# EFICIENCIA basada en rangos razonables
df['EFICIENCIA'] = (
    (df['INDICE_OCUPACIONAL'] > 0.7) &
    (df['LETALIDAD'] < 0.05) &
    (df['INDICE_ROTACION'] > df['INDICE_ROTACION'].median())
).astype(int)


# Revisar primeras filas
print(df[['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION', 'EFICIENCIA']].head())

# Contar cuántos 1 y 0 hay
Distribucion_Clases = df['EFICIENCIA'].value_counts()
Proporcion_Clases = df['EFICIENCIA'].value_counts(normalize=True) * 100

print("\nDistribución de Clases:")
print(Distribucion_Clases)

print("\nProporción de Clases:")
print(Proporcion_Clases)



## **Balanceo**

-Se utilizó SMOTE porque el conjunto de datos presenta un desbalance de clases (73%-27%) entre hospitales eficientes y no eficientes. Esta técnica genera nuevos ejemplos sintéticos de la clase minoritaria a partir de sus vecinos más cercanos, lo que permite equilibrar las clases sin eliminar información y mejorar el rendimiento de los modelos de clasificación, especialmente con variables numéricas continuas como las de este caso.

In [ ]:

X = df[['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']]
y = df['EFICIENCIA']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(y.value_counts())
print(y_train_res.value_counts())



## Tipo de datos de cada columna


In [ ]:
print(df.dtypes)

In [ ]:
df_numerico = df.select_dtypes(include=['number'])

## **Medidas Descriptivas**

In [ ]:
descripcion_estadistica = df_numerico.describe()
print(descripcion_estadistica)

## **Media**

In [ ]:
media = df_numerico.mean()
print(f"Media:\n{media}")

## **Medianas**

In [ ]:
mediana = df_numerico.median()
print(f"Mediana:\n{mediana}")

## **Moda**

In [ ]:
moda = df_numerico.mode().iloc[0]
print(f"Moda:\n{moda}")

## **Desviacion Estandar**

In [ ]:
desviacion_estandar = df_numerico.std()
print(f"Desviación Estándar:\n{desviacion_estandar}")

## **Percentiles**

In [ ]:
percentiles = df_numerico.quantile(q=[0.1, 0.25, 0.5, 0.75, 0.9])
print(f"Percentiles:\n{percentiles}")

In [ ]:
# Detección de valores atípicos usando el rango intercuartílico

Q1 = df_numerico.quantile(0.25)
Q3 = df_numerico.quantile(0.75)
IQR = Q3 - Q1

# Identificar outliers
outliers = ((df_numerico < (Q1 - 1.5 * IQR)) | (df_numerico > (Q3 + 1.5 * IQR))).sum()
print(f"Outliers detectados por columna:\n{outliers}")


# **1. b. Visualización de los datos**

In [ ]:
# Indice ocupacional
df['INDICE_OCUPACIONAL'] = pd.to_numeric(df['INDICE_OCUPACIONAL'], errors='coerce')

plt.hist(df['INDICE_OCUPACIONAL'].dropna(), bins=10, alpha=0.7, color='steelblue')
plt.xlabel('Índice Ocupacional')
plt.ylabel('Frecuencia')
plt.title('Distribución del Índice Ocupacional')
plt.show()

# Letalidad
df['LETALIDAD'] = pd.to_numeric(df['LETALIDAD'], errors='coerce')

plt.hist(df['LETALIDAD'].dropna(), bins=10, alpha=0.7, color='tomato')
plt.xlabel('Letalidad')
plt.ylabel('Frecuencia')
plt.title('Distribución de la Letalidad')
plt.show()

# Indice de rotacion
df['INDICE_ROTACION'] = pd.to_numeric(df['INDICE_ROTACION'], errors='coerce')

plt.hist(df['INDICE_ROTACION'].dropna(), bins=10, alpha=0.7, color='seagreen')
plt.xlabel('Índice de Rotación')
plt.ylabel('Frecuencia')
plt.title('Distribución del Índice de Rotación')
plt.show()

In [ ]:
# Boxplot de Índice Ocupacional por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'INDICE_OCUPACIONAL'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'INDICE_OCUPACIONAL'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Índice Ocupacional')
plt.title('Índice Ocupacional según Eficiencia')
plt.show()

# Boxplot de Letalidad por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'LETALIDAD'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'LETALIDAD'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Letalidad')
plt.title('Letalidad según Eficiencia')
plt.show()

# Boxplot de Índice de Rotación por eficiencia
plt.boxplot(
    [df.loc[df['EFICIENCIA'] == 0, 'INDICE_ROTACION'].dropna(),
     df.loc[df['EFICIENCIA'] == 1, 'INDICE_ROTACION'].dropna()],
    labels=['No Eficiente', 'Eficiente']
)
plt.ylabel('Índice de Rotación')
plt.title('Índice de Rotación según Eficiencia')
plt.show()

In [ ]:
# Asegurar tipo numérico
df['INDICE_OCUPACIONAL'] = pd.to_numeric(df['INDICE_OCUPACIONAL'], errors='coerce')
df['LETALIDAD'] = pd.to_numeric(df['LETALIDAD'], errors='coerce')

# Separar por eficiencia
eficiente = df[df['EFICIENCIA'] == 1]
no_eficiente = df[df['EFICIENCIA'] == 0]

plt.scatter(no_eficiente['INDICE_OCUPACIONAL'], no_eficiente['LETALIDAD'], alpha=0.5, label='No Eficiente', color='red')
plt.scatter(eficiente['INDICE_OCUPACIONAL'], eficiente['LETALIDAD'], alpha=0.5, label='Eficiente', color='green')
plt.xlabel('Índice Ocupacional')
plt.ylabel('Letalidad')
plt.title('Relación entre Índice Ocupacional y Letalidad según Eficiencia')
plt.legend()
plt.show()


# **1. c. Exploración, limpieza y transformación de datos**

Exploracion inicial

In [ ]:
# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
display(df.head())

# Información general del dataset
print("\nInformación general del dataset:")
print(df.info())

# Descripción estadística de las variables numéricas
print("\nDescripción estadística de las variables numéricas:")
display(df.describe().T)


# Revisión de valores únicos en columnas categóricas
print("\nValores únicos en columnas categóricas:")
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].nunique()} valores únicos")


Manejo de Valores Anomalos o Nulos

In [ ]:
# Contar valores nulos totales
print("Valores nulos totales antes de limpiar:")
print(df.isnull().sum().sum())

# Eliminar filas con valores nulos en variables relevantes
variables_relevantes = ['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']
df_limpio = df.dropna(subset=variables_relevantes)

print(f"\nFilas originales: {len(df)}, Filas después de eliminar nulos: {len(df_limpio)}")

# Detección de valores atípicos usando el rango intercuartílico (IQR)
for col in variables_relevantes:
    Q1 = df_limpio[col].quantile(0.25)
    Q3 = df_limpio[col].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df_limpio = df_limpio[(df_limpio[col] >= limite_inferior) & (df_limpio[col] <= limite_superior)]

print(f"Filas después de eliminar outliers: {len(df_limpio)}")

# Volver a calcular estadísticas con los datos limpios
print("\nDescripción estadística después de limpiar los datos:")
display(df_limpio.describe().T)


Se realiza la Tranformaciones

In [ ]:

# Copiar el dataset limpio
df_transformado = df_limpio.copy()

# Transformar variables categóricas en numéricas (Label Encoding)
columnas_categoricas = ['GLOSA_SSS', 'AREA_FUNCIONAL', 'ESTABLECIMIENTO']
le = LabelEncoder()
for col in columnas_categoricas:
    if col in df_transformado.columns:
        df_transformado[col] = le.fit_transform(df_transformado[col].astype(str))

print("\nColumnas categóricas transformadas a numéricas correctamente.")

# Estandarización de variables numéricas
columnas_numericas = ['INDICE_OCUPACIONAL', 'LETALIDAD', 'INDICE_ROTACION']
scaler = StandardScaler()
df_transformado[columnas_numericas] = scaler.fit_transform(df_transformado[columnas_numericas])

print("Variables numéricas estandarizadas correctamente.")

# Verificar el resultado final
print("\nVista general de los datos transformados:")
display(df_transformado.head())


# Clasificacion y análisis

## a. Aplicación de algoritmos de clasificación

In [ ]:
# Aplicación de algoritmos de clasificación:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

modelos = {
    'RandomForest': RandomForestClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42)
}

## b. Selección de modelos óptimos

Para seleccionar los modelos más óptimos se utiliza el análisis de la **curva ROC (Receiver Operating Characteristic)**, que permite comparar el rendimiento de los modelos en términos de la tasa de falsos positivos y verdaderos positivos.

**Métricas principales:**
- **Precisión, Recall, F1-Score:** Evaluación del desempeño de clasificación
- **Curva ROC y AUC:** Visualización del rendimiento del clasificador a diferentes umbrales
- **Validación cruzada (StratifiedKFold):** Garantiza proporciones equilibradas de clases en cada pliegue

El modelo con mayor AUC-ROC será seleccionado como el modelo óptimo para las etapas posteriores.


## c. Ajuste de hiperparámetros

El ajuste de hiperparámetros es fundamental para optimizar el rendimiento de los modelos. Se utilizan técnicas como GridSearchCV para búsqueda exhaustiva y RandomizedSearchCV para búsqueda aleatoria, combinadas con validación cruzada estratificada.

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, StratifiedKFold

param_grids = {
    'RandomForest': {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, None]},
    'AdaBoost': {'n_estimators': [50, 100, 200], 'learning_rate': [0.5, 1.0, 1.5]},
    'LogisticRegression': {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
}

best_estimators = {}
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

for name, model in modelos.items():
    print(f"\nAjustando hiperparámetros para: {name}")
    if name == 'LogisticRegression':
        search = RandomizedSearchCV(model, param_grids[name], n_iter=5, scoring='roc_auc', cv=cv, n_jobs=-1, random_state=42)
    else:
        search = GridSearchCV(model, param_grids[name], scoring='roc_auc', cv=cv, n_jobs=-1)
    search.fit(X_train, y_train)
    print(f"Mejores parámetros para {name}: {search.best_params_}")
    best_estimators[name] = search.best_estimator_

## d. Entrenamiento y validación de modelos

Los modelos se entrenan utilizando los datos balanceados con SMOTE y se validan en el conjunto de prueba. Se generan predicciones y se evalúan mediante múltiples métricas de clasificación: precisión, recall, F1-score, matriz de confusión y curva ROC.

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
results = {}
for name, best in best_estimators.items():
    y_pred = best.predict(X_test)
    # Calibrar probabilidades si es posible
    try:
        if hasattr(best, 'predict_proba'):
            calibrator = CalibratedClassifierCV(best, cv=3, method='sigmoid')
            calibrator.fit(X_train, y_train)
            prob_model = calibrator
        else:
            prob_model = best
    except Exception as e:
        print(f"Advertencia calibración {name}: {e}")
        prob_model = best
    y_proba = None
    if hasattr(prob_model, 'predict_proba'):
        y_proba = prob_model.predict_proba(X_test)[:, 1]
    elif hasattr(prob_model, 'decision_function'):
        y_proba = prob_model.decision_function(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    print(f"\n{name} - Accuracy: {acc:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f} | F1: {f1:.4f} | ROC AUC: {auc if auc is not None else 'N/A'}")
    print(classification_report(y_test, y_pred, zero_division=0))
    print('Confusion matrix:')
    print(confusion_matrix(y_test, y_pred))
    results[name] = {
        'model': best,
        'prob_model': prob_model,
        'y_proba': y_proba,
        'accuracy': acc,
        'precision': prec,
        'recall': rec,
        'f1': f1,
        'auc': auc
    }

## e. Evaluación de modelos

La evaluación se realiza utilizando las curvas ROC para comparar el rendimiento de los tres modelos. Se presentan reportes de clasificación detallados incluyendo precisión, recall, F1-score y matriz de confusión para cada modelo.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc as calc_auc
plt.figure(figsize=(10,7))
for name, info in results.items():
    prob = info.get('y_proba')
    if prob is not None:
        fpr, tpr, _ = roc_curve(y_test, prob)
        roc_auc = calc_auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.3f})")
plt.plot([0,1],[0,1],'k--', alpha=0.6)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Comparación de modelos')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

In [ ]:
for name in ['RandomForest', 'GradientBoosting', 'DecisionTree']:
    if name in results:
        model = results[name]['model']
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            fi = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
            print(f'\nImportancia de variables ({name}):')
            print(fi)
            fi.plot(kind='bar', title=f'Feature importance - {name}')
            plt.show()

## f. Predicciones

Se selecciona el modelo con mejor rendimiento (mayor AUC-ROC) para realizar predicciones en el conjunto de prueba. Se analiza cómo los diferentes factores como días de camas ocupadas, número de egresos e índices operacionales influyen en la predicción de eficiencia hospitalaria.

In [ ]:
# Seleccionar el mejor modelo según AUC
mejor_modelo = max(results.items(), key=lambda x: x[1]['auc'])[1]['model']
print("Mejor modelo seleccionado:", mejor_modelo.__class__.__name__)

# Realizar predicciones sobre el conjunto de prueba
y_pred = mejor_modelo.predict(X_test)

# Mostrar algunas predicciones junto a los valores reales
predicciones_df = X_test.copy()
predicciones_df['Real'] = y_test.values
predicciones_df['Predicho'] = y_pred
print(predicciones_df.head(10))

# Analizar la influencia de los factores
print("\nResumen de predicciones:")
print(predicciones_df.groupby(['Real', 'Predicho']).size().unstack(fill_value=0))

#### Análisis de la influencia de los factores en la eficiencia hospitalaria

Al analizar los resultados del modelo y la importancia de las variables, se observa que:

- **Índice Ocupacional**: Es el factor más relevante para predecir la eficiencia. Un mayor uso de camas respecto a las disponibles indica un uso óptimo de los recursos hospitalarios, lo que se asocia a hospitales eficientes.
- **Letalidad**: Tiene una influencia negativa en la eficiencia. Hospitales con menor proporción de fallecidos respecto al total de egresos tienden a ser clasificados como eficientes, reflejando una mejor calidad de atención.
- **Índice de Rotación**: También es importante. Un mayor índice de rotación implica que las camas se utilizan de manera dinámica, permitiendo atender a más pacientes y optimizando la gestión hospitalaria.

En resumen, los factores relacionados con la ocupación y gestión de camas, así como la calidad clínica (baja letalidad), son determinantes para que un hospital sea clasificado como eficiente según los modelos utilizados. Esto valida la utilidad de los modelos de clasificación para identificar oportunidades de mejora en la gestión hospitalaria.

### g. Preguntas

**i. ¿Qué conclusiones puedes obtener sobre los métodos utilizados?**  
Los métodos de clasificación aplicados permiten predecir con buena precisión la eficiencia hospitalaria. El uso de modelos de árbol y meta-algoritmos facilita la interpretación de los factores más relevantes y mejora la capacidad predictiva respecto a métodos tradicionales.

**ii. ¿Cuál recomendarías utilizar y por qué?**  
Recomendaría el uso de RandomForestClassifier, ya que combina buen rendimiento, robustez ante datos ruidosos y permite interpretar la importancia de las variables, facilitando la toma de decisiones basada en datos.

**iii. ¿Qué algoritmo de clasificación mostró el mejor rendimiento según las métricas utilizadas (precisión, recall, F1-score)?**  
Según las métricas obtenidas (precisión, recall, F1-score y AUC), el modelo RandomForestClassifier fue el que mostró el mejor rendimiento general en el conjunto de prueba.

**iv. ¿Cómo afectó el ajuste de hiperparámetros al rendimiento de los modelos?**  
El ajuste de hiperparámetros mediante GridSearchCV y RandomizedSearchCV permitió mejorar significativamente el rendimiento de los modelos, optimizando su capacidad de generalización y aumentando métricas como el AUC y el F1-score.

**v. ¿Qué ventaja se observa en el uso de algoritmos de clasificación para predecir la compatibilidad entre los participantes frente a los métodos tradicionales?**  
La principal ventaja es que los algoritmos de clasificación pueden manejar múltiples variables simultáneamente, detectar patrones complejos y ofrecer predicciones más precisas y objetivas, superando las limitaciones de los métodos tradicionales basados solo en reglas o umbrales simples.

### Selección y reducción de variables

Para optimizar el rendimiento y la interpretabilidad de los modelos, se aplican dos técnicas:

- **Selección de variables:** Se utiliza `SelectKBest` para seleccionar las variables más relevantes según la prueba estadística ANOVA F.
- **Reducción de variables:** Se utiliza `PCA` (Análisis de Componentes Principales) para reducir la dimensionalidad del conjunto de datos.

A continuación, se evalúa el impacto de estas técnicas en el rendimiento de los modelos de clasificación.

In [ ]:
from sklearn.ensemble import RandomForestClassifier  # Agrega esta línea

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA

# Selección de variables (SelectKBest)
selector = SelectKBest(score_func=f_classif, k=2)
X_train_sel = selector.fit_transform(X_train, y_train)
X_test_sel = selector.transform(X_test)
print("Variables seleccionadas (SelectKBest):", X_train.columns[selector.get_support()])

# Reducción de variables (PCA)
pca = PCA(n_components=2, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Evaluar impacto en RandomForest (como ejemplo)
modelo_rf = RandomForestClassifier(random_state=42)
modelo_rf.fit(X_train_sel, y_train)
y_pred_sel = modelo_rf.predict(X_test_sel)
acc_sel = accuracy_score(y_test, y_pred_sel)
print("Accuracy con SelectKBest:", acc_sel)

modelo_rf_pca = RandomForestClassifier(random_state=42)
modelo_rf_pca.fit(X_train_pca, y_train)
y_pred_pca = modelo_rf_pca.predict(X_test_pca)
acc_pca = accuracy_score(y_test, y_pred_pca)
print("Accuracy con PCA:", acc_pca)

### Conclusiones

**a. Comparación de resultados entre algoritmos de clasificación**  
Al comparar los resultados de los diferentes algoritmos aplicados (RandomForest, AdaBoost, LogisticRegression), se observa que RandomForestClassifier obtuvo el mejor desempeño general según las métricas de precisión, recall, F1-score y AUC. AdaBoost mostró un rendimiento competitivo, pero ligeramente inferior, mientras que LogisticRegression fue el más sencillo y menos robusto ante la complejidad de los datos.

**b. Efectividad de los modelos para predecir las decisiones**  
Los modelos de clasificación fueron efectivos para predecir la eficiencia hospitalaria, logrando altos valores de precisión y recall en el conjunto de prueba. Las predicciones se alinearon en gran medida con las decisiones esperadas, especialmente en el caso de RandomForest, que logró identificar correctamente la mayoría de los hospitales eficientes y no eficientes.

**c. Reflexión sobre enfoques y posibles mejoras**  
El enfoque basado en modelos de árbol (como RandomForest) resultó más adecuado para este contexto, ya que permite manejar relaciones no lineales y evaluar la importancia de cada variable. La incorporación de más variables relevantes (por ejemplo, recursos humanos, presupuesto, infraestructura) o el uso de técnicas avanzadas como el ensamblado de modelos o el ajuste fino de hiperparámetros podría mejorar aún más la capacidad predictiva. Además, explorar métodos de reducción de dimensionalidad o técnicas de selección de variables puede ayudar a simplificar los modelos sin perder precisión.